In [95]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold

In [96]:
df = pd.read_csv(r"C:\Users\Jason\Downloads\Fire_Incident_Dispatch_Data_20250408.csv")
print(df.shape)
total_rows = df.shape[0]
total_columns = df.shape[1]

C:\Users\Jason\AppData\Local\Temp\ipykernel_9600\2393140820.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r"C:\Users\Jason\Downloads\Fire_Incident_Dispatch_Data_20250408.csv")


(10275092, 29)


In [97]:
# Dropping unnecessary columns that contained no useful information
df.drop(columns=['STARFIRE_INCIDENT_ID', 'ZIPCODE', 'POLICEPRECINCT', 'CITYCOUNCILDISTRICT', 'COMMUNITYDISTRICT', 'COMMUNITYSCHOOLDISTRICT', 'CONGRESSIONALDISTRICT', 'ALARM_BOX_NUMBER', 'ALARM_BOX_LOCATION', 'INCIDENT_CLASSIFICATION', 'ALARM_LEVEL_INDEX_DESCRIPTION', 'HIGHEST_ALARM_LEVEL', 'VALID_DISPATCH_RSPNS_TIME_INDC'], inplace=True)
#Dropping all columns with DateTime data types
df.drop(columns=['FIRST_ASSIGNMENT_DATETIME', 'FIRST_ACTIVATION_DATETIME', 'FIRST_ON_SCENE_DATETIME', 'INCIDENT_CLOSE_DATETIME'], inplace=True)
missing_counts = df.isnull().sum()
missing_counts = missing_counts[missing_counts > 0]
print(missing_counts)

ALARM_BOX_BOROUGH                     2
DISPATCH_RESPONSE_SECONDS_QY        936
INCIDENT_RESPONSE_SECONDS_QY     408002
INCIDENT_TRAVEL_TM_SECONDS_QY    408034
ENGINES_ASSIGNED_QUANTITY          1584
LADDERS_ASSIGNED_QUANTITY          1584
OTHER_UNITS_ASSIGNED_QUANTITY      1584
dtype: int64


In [98]:
# We can see that the rows with missing values only account for around 4% of the total rows, so we can drop them without losing too much data.
df.dropna(inplace=True)
print(df.shape)
# Remapping the values in VALID_INCIDENT_RSPNS_TIME_INDC
df['VALID_INCIDENT_RSPNS_TIME_INDC'] = df['VALID_INCIDENT_RSPNS_TIME_INDC'].map({'Y': 1, 'N': 0})

(9866972, 12)


In [99]:
# Checking to see if the ALARM_BOX_BOROUGH and INCIDENT_BOROUGH columns are equal enough to be used as a single column, they are comepletley equal, so they will be combined
print((df['ALARM_BOX_BOROUGH'] == df['INCIDENT_BOROUGH']).value_counts())
df.drop(columns=['ALARM_BOX_BOROUGH'], inplace=True)

True    9866972
Name: count, dtype: int64


In [100]:
# Convert string to datetime
df['INCIDENT_DATETIME'] = pd.to_datetime(df['INCIDENT_DATETIME'], format="%m/%d/%Y %I:%M:%S %p")

# Extract hour in 24-hour format
df['INCIDENT_HOUR'] = df['INCIDENT_DATETIME'].dt.hour
df.drop(columns=['INCIDENT_DATETIME'], inplace=True)
df.head()

,INCIDENT_BOROUGH,ALARM_SOURCE_DESCRIPTION_TX,INCIDENT_CLASSIFICATION_GROUP,DISPATCH_RESPONSE_SECONDS_QY,VALID_INCIDENT_RSPNS_TIME_INDC,INCIDENT_RESPONSE_SECONDS_QY,INCIDENT_TRAVEL_TM_SECONDS_QY,ENGINES_ASSIGNED_QUANTITY,LADDERS_ASSIGNED_QUANTITY,OTHER_UNITS_ASSIGNED_QUANTITY,INCIDENT_HOUR
0,RICHMOND / STATEN ISLAND,Phone,NonMedical Emergencies,40,1,397,357,1.0,1.0,0.0,0
1,BRONX,PD Link/Medical,Medical MFAs,10,1,260,250,1.0,0.0,0.0,0
2,BROOKLYN,PD Link/Medical,Medical Emergencies,7,1,146,139,1.0,0.0,0.0,0
3,QUEENS,Phone,NonMedical MFAs,52,1,327,275,3.0,2.0,1.0,0
4,QUEENS,Private Fire Alarm,NonMedical Emergencies,112,1,391,279,1.0,1.0,1.0,0


In [101]:
df.groupby('INCIDENT_HOUR').count()

,INCIDENT_BOROUGH,ALARM_SOURCE_DESCRIPTION_TX,INCIDENT_CLASSIFICATION_GROUP,DISPATCH_RESPONSE_SECONDS_QY,VALID_INCIDENT_RSPNS_TIME_INDC,INCIDENT_RESPONSE_SECONDS_QY,INCIDENT_TRAVEL_TM_SECONDS_QY,ENGINES_ASSIGNED_QUANTITY,LADDERS_ASSIGNED_QUANTITY,OTHER_UNITS_ASSIGNED_QUANTITY
INCIDENT_HOUR,,,,,,,,,,
0,328378,328378,328378,328378,328378,328378,328378,328378,328378,328378
1,271353,271353,271353,271353,271353,271353,271353,271353,271353,271353
2,227686,227686,227686,227686,227686,227686,227686,227686,227686,227686
3,201021,201021,201021,201021,201021,201021,201021,201021,201021,201021
4,191727,191727,191727,191727,191727,191727,191727,191727,191727,191727
5,192771,192771,192771,192771,192771,192771,192771,192771,192771,192771
6,234246,234246,234246,234246,234246,234246,234246,234246,234246,234246
7,310532,310532,310532,310532,310532,310532,310532,310532,310532,310532
8,384049,384049,384049,384049,384049,384049,384049,384049,384049,384049


In [ ]:
def hour_to_ampm(h):
    suffix = "AM" if h < 12 or h == 24 else "PM"
    hour12 = h % 12
    if hour12 == 0:
        hour12 = 12
    return f"{hour12} {suffix}"


def assign_optimal_hour_groups_by_counts(df, hour_col='INCIDENT_HOUR', group_col='HOUR GROUP'):

    def get_hour_groups(start):
        hours = [(start + i) % 24 for i in range(24)]
        return [hours[i:i+4] for i in range(0, 24, 4)]

    # Precompute value counts of each hour
    hour_counts = df[hour_col].value_counts().reindex(range(24), fill_value=0)

    best_std = np.inf
    best_groups = None

    for start in range(24):
        groups = get_hour_groups(start)
        group_totals = [hour_counts[group].sum() for group in groups]
        std = np.std(group_totals, ddof=0)
        print(f"Start hour: {start:02d} -> Group totals: {group_totals} -> Std Dev: {std:.2f}")
        if std < best_std:
            best_std = std
            best_groups = groups

    # Create mapping from hour to group index + label
    hour_to_group = {}
    hour_to_label = {}

    for i, group in enumerate(best_groups):
        for hour in group:
            hour_to_group[hour] = i
            start_hour = group[0]
            end_hour = group[-1]
            label = f"{hour_to_ampm(start_hour)} - {hour_to_ampm((end_hour + 1) % 24)}"
            hour_to_label[hour] = label

    df[group_col] = df[hour_col].map(hour_to_group)
    df[f"{group_col} LABEL"] = df[hour_col].map(hour_to_label)

    print(f"\nBest grouping: {best_groups} with Std Dev: {best_std:.2f}")

    return df, best_groups

In [112]:
df, best_groups = assign_optimal_hour_groups_by_counts(df)
print("Best grouping:", best_groups)
df.head()

Start hour: 00 -> Group totals: [np.int64(1028438), np.int64(929276), np.int64(1769984), np.int64(2094585), np.int64(2228041), np.int64(1816648)] -> Std Dev: 496580.90
Start hour: 01 -> Group totals: [np.int64(891787), np.int64(1121598), np.int64(1888006), np.int64(2141476), np.int64(2194627), np.int64(1629478)] -> Std Dev: 491451.77
Start hour: 02 -> Group totals: [np.int64(813205), np.int64(1360550), np.int64(1971646), np.int64(2196427), np.int64(2102430), np.int64(1422714)] -> Std Dev: 490274.72
Start hour: 03 -> Group totals: [np.int64(819765), np.int64(1590073), np.int64(2041233), np.int64(2226896), np.int64(1969679), np.int64(1219326)] -> Std Dev: 494342.43
Start hour: 04 -> Group totals: [np.int64(929276), np.int64(1769984), np.int64(2094585), np.int64(2228041), np.int64(1816648), np.int64(1028438)] -> Std Dev: 496580.90
Start hour: 05 -> Group totals: [np.int64(1121598), np.int64(1888006), np.int64(2141476), np.int64(2194627), np.int64(1629478), np.int64(891787)] -> Std Dev: 49

,INCIDENT_BOROUGH,ALARM_SOURCE_DESCRIPTION_TX,INCIDENT_CLASSIFICATION_GROUP,DISPATCH_RESPONSE_SECONDS_QY,VALID_INCIDENT_RSPNS_TIME_INDC,INCIDENT_RESPONSE_SECONDS_QY,INCIDENT_TRAVEL_TM_SECONDS_QY,ENGINES_ASSIGNED_QUANTITY,LADDERS_ASSIGNED_QUANTITY,OTHER_UNITS_ASSIGNED_QUANTITY,INCIDENT_HOUR,HOUR GROUP,HOUR GROUP LABEL
0,RICHMOND / STATEN ISLAND,Phone,NonMedical Emergencies,40,1,397,357,1.0,1.0,0.0,0,5,10 PM - 2 AM
1,BRONX,PD Link/Medical,Medical MFAs,10,1,260,250,1.0,0.0,0.0,0,5,10 PM - 2 AM
2,BROOKLYN,PD Link/Medical,Medical Emergencies,7,1,146,139,1.0,0.0,0.0,0,5,10 PM - 2 AM
3,QUEENS,Phone,NonMedical MFAs,52,1,327,275,3.0,2.0,1.0,0,5,10 PM - 2 AM
4,QUEENS,Private Fire Alarm,NonMedical Emergencies,112,1,391,279,1.0,1.0,1.0,0,5,10 PM - 2 AM


In [113]:
df.groupby('HOUR GROUP LABEL').count()

,INCIDENT_BOROUGH,ALARM_SOURCE_DESCRIPTION_TX,INCIDENT_CLASSIFICATION_GROUP,DISPATCH_RESPONSE_SECONDS_QY,VALID_INCIDENT_RSPNS_TIME_INDC,INCIDENT_RESPONSE_SECONDS_QY,INCIDENT_TRAVEL_TM_SECONDS_QY,ENGINES_ASSIGNED_QUANTITY,LADDERS_ASSIGNED_QUANTITY,OTHER_UNITS_ASSIGNED_QUANTITY,INCIDENT_HOUR,HOUR GROUP
HOUR GROUP LABEL,,,,,,,,,,,,
10 AM - 2 PM,1971646,1971646,1971646,1971646,1971646,1971646,1971646,1971646,1971646,1971646,1971646,1971646
10 PM - 2 AM,1422714,1422714,1422714,1422714,1422714,1422714,1422714,1422714,1422714,1422714,1422714,1422714
2 AM - 6 AM,813205,813205,813205,813205,813205,813205,813205,813205,813205,813205,813205,813205
2 PM - 6 PM,2196427,2196427,2196427,2196427,2196427,2196427,2196427,2196427,2196427,2196427,2196427,2196427
6 AM - 10 AM,1360550,1360550,1360550,1360550,1360550,1360550,1360550,1360550,1360550,1360550,1360550,1360550
6 PM - 10 PM,2102430,2102430,2102430,2102430,2102430,2102430,2102430,2102430,2102430,2102430,2102430,2102430


In [ ]:
df.groupby('INCIDENT_BOROUGH').count()

,ALARM_SOURCE_DESCRIPTION_TX,INCIDENT_CLASSIFICATION_GROUP,DISPATCH_RESPONSE_SECONDS_QY,VALID_INCIDENT_RSPNS_TIME_INDC,INCIDENT_RESPONSE_SECONDS_QY,INCIDENT_TRAVEL_TM_SECONDS_QY,ENGINES_ASSIGNED_QUANTITY,LADDERS_ASSIGNED_QUANTITY,OTHER_UNITS_ASSIGNED_QUANTITY,INCIDENT_HOUR,hour,hour_group
INCIDENT_BOROUGH,,,,,,,,,,,,
BRONX,1996904,1996904,1996904,1996904,1996904,1996904,1996904,1996904,1996904,1996904,1996904,1996904
BROOKLYN,2808691,2808691,2808691,2808691,2808691,2808691,2808691,2808691,2808691,2808691,2808691,2808691
MANHATTAN,2591202,2591202,2591202,2591202,2591202,2591202,2591202,2591202,2591202,2591202,2591202,2591202
QUEENS,1966132,1966132,1966132,1966132,1966132,1966132,1966132,1966132,1966132,1966132,1966132,1966132
RICHMOND / STATEN ISLAND,504043,504043,504043,504043,504043,504043,504043,504043,504043,504043,504043,504043


In [ ]:
value_counts = df['ALARM_SOURCE_DESCRIPTION_TX'].value_counts()
valid_categories = value_counts[value_counts >= 5000].index
df = df[df['ALARM_SOURCE_DESCRIPTION_TX'].isin(valid_categories)]
df.groupby('ALARM_SOURCE_DESCRIPTION_TX').count()
print(f"Percentage of remaining rows: {df.shape[0] / total_rows * 100:.2f}%")

Percentage of remaining rows: 95.97%


In [ ]:
df.groupby('INCIDENT_CLASSIFICATION_GROUP').count()

,INCIDENT_BOROUGH,ALARM_SOURCE_DESCRIPTION_TX,DISPATCH_RESPONSE_SECONDS_QY,VALID_INCIDENT_RSPNS_TIME_INDC,INCIDENT_RESPONSE_SECONDS_QY,INCIDENT_TRAVEL_TM_SECONDS_QY,ENGINES_ASSIGNED_QUANTITY,LADDERS_ASSIGNED_QUANTITY,OTHER_UNITS_ASSIGNED_QUANTITY,INCIDENT_HOUR,hour,hour_group
INCIDENT_CLASSIFICATION_GROUP,,,,,,,,,,,,
Medical Emergencies,4391908,4391908,4391908,4391908,4391908,4391908,4391908,4391908,4391908,4391908,4391908,4391908
Medical MFAs,67780,67780,67780,67780,67780,67780,67780,67780,67780,67780,67780,67780
NonMedical Emergencies,4149412,4149412,4149412,4149412,4149412,4149412,4149412,4149412,4149412,4149412,4149412,4149412
NonMedical MFAs,427417,427417,427417,427417,427417,427417,427417,427417,427417,427417,427417,427417
NonStructural Fires,305901,305901,305901,305901,305901,305901,305901,305901,305901,305901,305901,305901
Structural Fires,518838,518838,518838,518838,518838,518838,518838,518838,518838,518838,518838,518838


In [ ]:
# Target Encoding for Categorical Variables using Stratified K-Folds

def target_encode(df, col, target='VALID_INCIDENT_RSPNS_TIME_INDC', n_splits=10, alpha=10):

    df = df.copy()
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    encoded = pd.Series(index=df.index, dtype=float)
    
    global_mean = df[target].mean()

    for train_idx, val_idx in skf.split(df, df[target]):
        train, val = df.iloc[train_idx], df.iloc[val_idx]

        # Compute smoothed means
        category_stats = train.groupby(col)[target].agg(['mean', 'count'])
        smooth = (category_stats['count'] * category_stats['mean'] + alpha * global_mean) / (category_stats['count'] + alpha)

        # Map to validation fold
        encoded.iloc[val_idx] = val[col].map(smooth).fillna(global_mean)

    return encoded

In [ ]:
print(df.shape)
df.head()

(9861256, 13)


,INCIDENT_BOROUGH,ALARM_SOURCE_DESCRIPTION_TX,INCIDENT_CLASSIFICATION_GROUP,DISPATCH_RESPONSE_SECONDS_QY,VALID_INCIDENT_RSPNS_TIME_INDC,INCIDENT_RESPONSE_SECONDS_QY,INCIDENT_TRAVEL_TM_SECONDS_QY,ENGINES_ASSIGNED_QUANTITY,LADDERS_ASSIGNED_QUANTITY,OTHER_UNITS_ASSIGNED_QUANTITY,INCIDENT_HOUR,hour,hour_group
0,RICHMOND / STATEN ISLAND,Phone,NonMedical Emergencies,40,1,397,357,1.0,1.0,0.0,0,0,22-1
1,BRONX,PD Link/Medical,Medical MFAs,10,1,260,250,1.0,0.0,0.0,0,0,22-1
2,BROOKLYN,PD Link/Medical,Medical Emergencies,7,1,146,139,1.0,0.0,0.0,0,0,22-1
3,QUEENS,Phone,NonMedical MFAs,52,1,327,275,3.0,2.0,1.0,0,0,22-1
4,QUEENS,Private Fire Alarm,NonMedical Emergencies,112,1,391,279,1.0,1.0,1.0,0,0,22-1


In [ ]:
# Target Encoding for categorical variables
df['Borough num'] = target_encode(df, 'INCIDENT_BOROUGH')
df['Alarm Source num'] = target_encode(df, 'ALARM_SOURCE_DESCRIPTION_TX')
df['Incident Classification num'] = target_encode(df, 'INCIDENT_CLASSIFICATION_GROUP')
df['Time num'] = target_encode(df, 'HOUR GROUP LABEL')

# Computing cleaning metrics
cleaned_rows = df.shape[0]
cleaned_columns = df.shape[1]
print((cleaned_rows * cleaned_columns) / (total_rows * total_columns) * 100)


52.950309014432584


In [ ]:
# Renaming columns to make them easier to read and work with
df.rename(columns={'INCIDENT_BOROUGH': 'Borough', 'ALARM_SOURCE_DESCRIPTION_TX': 'Alarm Source', 'INCIDENT_CLASSIFICATION_GROUP': 'Incident Classification', 'DISPATCH_RESPONSE_SECONDS_QY': 'Dispatch Response Time', 'VALID_INCIDENT_RSPNS_TIME_INDC': 'Valid Response Time', 'INCIDENT_RESPONSE_SECONDS_QY': 'Incident Response Time', 'INCIDENT_TRAVEL_TM_SECONDS_QY': 'Incident Travel Time', 'ENGINES_ASSIGNED_QUANTITY': 'Engines Assigned', 'LADDERS_ASSIGNED_QUANTITY': 'Ladders Assigned', 'OTHER_UNITS_ASSIGNED_QUANTITY': 'Other Units Assigned', 'HOUR GROUP LABEL' : 'Time'}, inplace=True)
order = ['Borough', 'Borough num', 'Time', 'Time num',  'Alarm Source', 'Alarm Source num', 'Incident Classification', 'Incident Classification num', 'Dispatch Response Time', 'Incident Response Time', 'Incident Travel Time', 'Engines Assigned', 'Ladders Assigned', 'Other Units Assigned', 'Valid Response Time']
df = df[order]
df.head()

,Borough,Borough num,Alarm Source,Alarm Source num,Incident Classification,Incident Classification num,Dispatch Response Time,Incident Response Time,Incident Travel Time,Engines Assigned,Ladders Assigned,Other Units Assigned,Valid Response Time
0,RICHMOND / STATEN ISLAND,0.864650,Phone,0.957787,NonMedical Emergencies,0.951872,40,397,357,1.0,1.0,0.0,1
1,BRONX,0.829616,PD Link/Medical,0.687040,Medical MFAs,0.879852,10,260,250,1.0,0.0,0.0,1
2,BROOKLYN,0.860628,PD Link/Medical,0.687040,Medical Emergencies,0.695660,7,146,139,1.0,0.0,0.0,1
3,QUEENS,0.850763,Phone,0.957787,NonMedical MFAs,0.955968,52,327,275,3.0,2.0,1.0,1
4,QUEENS,0.850645,Private Fire Alarm,0.985807,NonMedical Emergencies,0.951918,112,391,279,1.0,1.0,1.0,1


In [ ]:
#pd.DataFrame(df).to_csv(r"C:\Users\Jason\Downloads\CleanedFireIncidentData.csv")